In [93]:
%reset -f

In [94]:
import flax.linen as nn
import jax
import jax.numpy as jnp
import jax.random as jr

## Data

Des dimensions fixées:

In [95]:
INP_DIM = 2
OUT_DIM = 3

Une fonction au pif. On va essayer d'ajuster un réseau de neurone à cette fonction.

In [96]:
def target_fn(inpV):
    outV0=jnp.sin(5*inpV[:,0]) * jnp.cos(8*inpV[:,1])
    outV1=jnp.sin(2*inpV[:,0]) + jnp.cos(4*inpV[:,1])**2
    outV2=jnp.sin(2*inpV[:,0]) - jnp.cos(7*inpV[:,1])*4
    return jnp.stack([outV0,outV1,outV2],axis=1)


target_fn(jnp.zeros([7,INP_DIM])).shape

## Définition du modèle



In [97]:

class SimpleNN(nn.Module):
    out_dim: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=128)(x)
        x = nn.relu(x)
        x = nn.Dense(features=64)(x)
        x = nn.relu(x)
        return nn.Dense(features=self.out_dim)(x)


dummy_input = jnp.ones((1, INP_DIM)) # Assuming flattened MNIST images
model = SimpleNN(OUT_DIM)
params = model.init(jr.key(0), dummy_input)
print(jax.tree.map(jnp.shape, params))
print("Model initialized successfully.")

## Définition de l'optimiseur



In [98]:
import optax

learning_rate = 0.01
optimizer = optax.adam(learning_rate)

## Définition de la fonction d'entraînement



In [99]:
@jax.jit
def loss_fn(params,inpV, outV_true):
    outV_pred = model.apply(params, inpV)
    return jnp.mean(jnp.square(outV_pred - outV_true))

In [100]:
@jax.jit
def train_step(params, opt_state, inpV, outV_true):
    """Performs a single training step."""
    loss_value, grads = jax.value_and_grad(loss_fn)(params,inpV, outV_true)

    updates, new_opt_state = optimizer.update(grads, opt_state, params)

    new_params = jax.tree.map(lambda x,y:x+y,params,updates)
    #ou bien:
    #new_params = optax.apply_updates(params, updates)

    return new_params, new_opt_state, loss_value

Note: l'optimiseur a lui aussi ses variables propres. L'ensemble de ses variables est appelé 'state'.

Dans les autres lib on utiliserais un attribut `optimizer.state` que l'on mettrait à jour de manière `inplace` (et caché).

En JAX on veut coder des fonctions pures. Cela donne la syntaxe ci-dessus, qui ne cache rien !

## Boucle d'entrainement

In [101]:
# Step 1: Initialize the optimizer state
opt_state = optimizer.init(params)

batch_size = 32
rkey=jr.key(0)
losses=[]

for step in range(1000):
    rkey, subkey = jr.split(rkey)
    inV=jr.uniform(subkey,(batch_size,INP_DIM))
    outV_true=target_fn(inV)
    params, opt_state, loss = train_step(params, opt_state, inV, outV_true)
    losses.append(loss)
    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss}")

In [102]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots()
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.set_yscale("log")
ax.plot(losses);

## Défi prog.

Transformer ce programme pour qu'il fonctionne avec `INP_DIM=2` et `OUT_DIM=2`.

Changez notamment la `target_fn`. Illustrez l'apprentissage optenu en traçant en niveau de couleur la `target_fn` à côté de la fonction prédite: `inp->model(params,inp)`